In [1]:
import pandas as pd
from datetime import datetime, date, timedelta
import requests
from pytz import timezone

In [2]:
def tomorrows_flight_arrivals(icao_list):

  today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
  tomorrow = (today + timedelta(days=1))

  list_for_df = []

  for icao in icao_list:
    times = [["00:00","11:59"],["12:00","23:59"]]

    for time in times:
      url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
      querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false"}
      headers = {
	       "X-RapidAPI-Key": "fede0c7146msh1bb3d4ba98e89a5p1df66ajsn5f7e89b742e5",
	       "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"}

      response = requests.request("GET", url, headers=headers, params=querystring)
      flights_json = response.json()

      for flight in flights_json['arrivals']:
        flights_dict = {}
        flights_dict['arrival_icao'] = icao
        # .get() is another way of ensuring our code doesn't break
        # in the previous 2 notebooks you learnt about 'if' (cities) and 'try/except' (weather)
        # .get() works similar, it will get the text if possible, if there is no text a None value will be inserted instead
        flights_dict['arrival_time'] = flight['arrival'].get('scheduledTimeLocal', None)
        flights_dict['arrival_terminal'] = flight['arrival'].get('terminal', None)
        flights_dict['departure_city'] = flight['departure']['airport'].get('name', None)
        flights_dict['departure_icao'] = flight['departure']['airport'].get('icao', None)
        flights_dict['departure_time_local'] = flight['departure'].get('scheduledTimeLocal', None)
        flights_dict['airline'] = flight['airline'].get('name', None)
        flights_dict['flight_num'] = flight.get('number', None)
        flights_dict['data_retrieved_on'] = datetime.now().astimezone(timezone('Europe/Berlin')).date()
        list_for_df.append(flights_dict)

  return pd.DataFrame(list_for_df)

In [7]:
icaos = ['EGLL']
flights_df=tomorrows_flight_arrivals(icaos)


,arrival_icao,arrival_time_local,arrival_terminal,departure_city,departure_icao,departure_time_local,airline,flight_number,data_retrieved_on
0,EGLL,2022-10-08 05:00+01:00,3,Jo'anna,FAOR,2022-10-07 18:45+02:00,Virgin Atlantic,VS 450,2022-10-07
1,EGLL,2022-10-08 05:30+01:00,5,Singapore,WSSS,2022-10-07 22:35+08:00,British Airways,BA 16,2022-10-07
2,EGLL,2022-10-08 05:30+01:00,5,Jo'anna,FAOR,2022-10-07 19:20+02:00,British Airways,BA 56,2022-10-07
3,EGLL,2022-10-08 05:40+01:00,5,Riyadh,OERK,2022-10-08 00:35+03:00,British Airways,BA 262,2022-10-07
4,EGLL,2022-10-08 05:40+01:00,3,Hong Kong,VHHH,2022-10-07 23:00+08:00,Cathay Pacific,CX 251,2022-10-07
...,...,...,...,...,...,...,...,...,...
527,EGLL,2022-10-08 22:00+01:00,5,Prague,LKPR,2022-10-08 21:00+02:00,British Airways,BA 857,2022-10-07
528,EGLL,2022-10-08 22:25+01:00,2,Istanbul,LTFM,2022-10-08 20:20+03:00,Turkish,TK 1987,2022-10-07
529,EGLL,2022-10-08 22:00+01:00,2,Lisbon,LPPT,2022-10-08 19:20+01:00,TAP Air Portugal,TP 1358,2022-10-07
530,EGLL,2022-10-08 22:55+01:00,2,Lisbon,LPPT,2022-10-08 20:15+01:00,TAP Air Portugal,TP 1366,2022-10-07


In [ ]:
flights= pd.DataFrame(flights_df, columns=['flight_num','departure_icao','arrival_icao','arrival_time']))